In [67]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional 
import numpy as np
INITIAL_BALANCE = 5000

In [68]:
df_xau = pd.read_csv("Files/backtest_formateado_XAU_M5.csv")
df_xau_2020 = pd.read_csv("Files/backtest_formateado_XAU_M5_2020-2025.csv",
                            parse_dates=["Time Entry","Time Exit"],
                            date_format="%Y.%m.%d %H:%M:%S")

df = pd.read_excel(r"Files/the5ers_bots.xlsx",sheet_name="All")

#Rellenando na con ceros 
df.fillna(0,inplace=True)


#Colocando comment en operaciones manuales
df.loc[df["Comment"]==0,"Comment"] = "Manual"

#Resetando indice
df.reset_index(inplace=True,drop=True)

#Rename a dos columnas que se llamaban iguales
df.rename(columns={"Price":"Price Entry", "Price.1":"Price Exit"},inplace=True)
df.rename(columns={"Time":"Time Entry","Time.1":"Time Exit"},inplace=True)

#Nueva columna con el balance 
# df["Balance"] = INITIAL_BALANCE + (df["Profit"]+df["Commission"]+df["Swap"]).cumsum() 



# Funciones

In [69]:
def balance_in_R(data:pd.DataFrame,RISK:float = 0.01):
    
    # * Formula: Balance_n = Balance_0 * PRODUCTORIO(1 + R_n* RISK) 
    
    #Limpio "R(usd)" que no me interesa
    if "R(usd)" in data.columns:
        data = data.drop(columns="R(usd)")

    #Interior del Productorio
    data["growth"] = 1+ data["R-multiple"]*RISK
    
    #Toda la Formula
    data["Balance_with_R"] = (INITIAL_BALANCE *data["growth"].cumprod()).round(2)
    
    
    return data[data.columns[-6:]]

# Cosas Varias

In [70]:
from logic import BotAnalyzer as bt
import plotly.express as px
import plotly.io as pio


In [71]:
fiveers = bt(df)
fiveers.add_R()

xau = bt(df_xau_2020)
xau.add_R()

xau.summary

xau.df.shape

(566, 16)

In [72]:
xau.trim_per_extreme_values(0.01)

xau.df["R_equity"] = bt.equity_R_curve(xau.df)


In [73]:

mon = xau.df.groupby(xau.df["Time Entry"].dt.to_period("M"))["R_equity"]


fig  = px.line(mon)

ValueError: Plotly Express cannot process wide-form data with columns of different type.